<h1><center> Text mining part </center></h1>

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
add_page = requests.get('https://auto.bazos.cz/inzerat/157012769/vw-golf-7-20tdi-110kw-107tkm-plna-zaruka-2-roky-zdarma.php')

In [3]:
soup_add = BeautifulSoup(add_page.text, 'html')

In [4]:
add = soup_add.find('div', {'class':'popisdetail'}).get_text()
print(add)

S tímhle autem můžete rovnou vyrazit na cesty. Vybrali jsme ho pečlivě přímo u autorizovaného dealera Volkswagen. Zkontrolovali vlastním servisem. A vybavili širokou dvouletou zárukou. Doveze vás kamkoliv stejně dobře jako nové. Jen za citelně lepší cenu.

JEZDĚTE V KLIDU - plnou dvouletou záruku máte teď k autu zdarma - Akce platí do konce prázdnin

Volkswagen Golf 7 Comfortline 2.0 TDI 110kW se zárukou 2 roky

Cena 428 000 Kč
DPH si můžete odečíst.

Po 1. majiteli
VIN: WVWZZZAUZJP076699
Registrace: 2/2018
Najeto: 107 604 km
Emise: Plní nejmodernější emisní normu EURO 6 Poznámka servisu: servisováno ve VW, STK do 2023, servisní knížka k dispozici, servisováno ve Volkswagen podle předepsaných intervalů.

VÝBAVA:
6 rychlostních stupňů, ACC adaptivní tempomat, Asistent nouzového brzdění, Asistent rozpoznávání únavy, Bezdrátová nabíječka mobilu, Climatronic 2-zónová, Čalounění sedadel Alcantara&Textil, Dešťový senzor, El. vyhřívaná zpětná zrcátka, ERGO masážní sedadlo řidiče, Funkce Auto 

In [11]:
def remove_spaces(text):
    return text.replace(" ", "")

In [17]:
add = remove_spaces(add)
print(add)

Stímhleautemmůžeterovnouvyrazitnacesty.VybralijsmehopečlivěpřímouautorizovanéhodealeraVolkswagen.Zkontrolovalivlastnímservisem.Avybaviliširokoudvouletouzárukou.Dovezeváskamkolivstejnědobřejakonové.Jenzacitelnělepšícenu.

JEZDĚTEVKLIDU-plnoudvouletouzárukumáteteďkautuzdarma-Akceplatídokonceprázdnin

VolkswagenGolf7Comfortline2.0TDI110kWsezárukou2roky

Cena428000Kč
DPHsimůžeteodečíst.

Po1.majiteli
VIN:WVWZZZAUZJP076699
Registrace:2/2018
Najeto:107604km
Emise:PlnínejmodernějšíemisnínormuEURO6Poznámkaservisu:servisovánoveVW,STKdo2023,servisníknížkakdispozici,servisovánoveVolkswagenpodlepředepsanýchintervalů.

VÝBAVA:
6rychlostníchstupňů,ACCadaptivnítempomat,Asistentnouzovéhobrzdění,Asistentrozpoznáváníúnavy,Bezdrátovánabíječkamobilu,Climatronic2-zónová,ČalouněnísedadelAlcantara&Textil,Dešťovýsenzor,El.vyhřívanázpětnázrcátka,ERGOmasážnísedadlořidiče,FunkceAutoHold,FULLLEDsvětlomety,Isofix,KontrolatrakceASR,Koženéčalouněnísedadel,LEDdennísvícení,MlhovésvětlometysfunkcíCorner,NavigaceDISCOVE

In [14]:
def get_numbers_from_text(text):
    import re
    text = remove_spaces(text)
    pattern = '[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?'
    list_of_numbers = re.findall(pattern, text)
    return list_of_numbers

In [27]:
all_numbers = get_numbers_from_text(add)
print(all_numbers)
type(all_numbers[0])

['7', '2.0', '110', '2', '428000', '1.', '076699', '2', '2018', '107604', '6', '2023', '6', '2', '117', '9000', '+420735127096', '+420284828188', '8']


str

In [35]:
def find_years(numbers):
    numbers = [x for x in numbers if (float(x) > 1980) and (float(x) < 2030)]
    return numbers

In [43]:
all_years = find_years(all_numbers)
print(all_years)

['2018', '2023']


In [40]:
type(all_years[0])

str

In [67]:
def find_km(numbers):
    numbers = [x for x in numbers if (float(x) > 3000) and (float(x) < 500000)]
    return numbers

In [68]:
test = find_km(all_numbers)
print(test)

['428000', '076699', '107604', '9000']


In [69]:
def get_context(text, list_of_tokens, year_dictionary, km_dictionary, context_span=20):
    import re
    context = []
    year = 'No match'
    km = 'No match'
    for token in find_years(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in year_dictionary:
                year_find = [m.start() for m in re.finditer(s, substring)]
                if len(year_find) > 0:
                    year = token
    for token in find_km(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in km_dictionary:
                km_find = [m.start() for m in re.finditer(s, substring)]
                if len(km_find) > 0:
                    km = token
    return [year, km] #context
year_dict = ['egistr', 'r.v', 'vyrob', 'výrob', 'prov']
km_dict = ['km', 'Km', 'KM', 'ilomet', 'ajet', 'ájez', 'achom', 'atoč']
context_got = get_context(add, all_numbers, year_dict, km_dict)
print(context_got)

['2018', '107604']


In [52]:
def get_context(text, list_of_tokens, context_span=20):
    import re
    context = []
    for token in list_of_tokens:
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            
            if left_index == 0:
                substring = ' '.join(substring.split(' ')[:-1])
            elif right_index == len(text):
                substring = ' '.join(substring.split(' ')[1:])
            else:
                substring = ' '.join(substring.split(' ')[1:-1])
            context.append(substring)
    return context
context_got = get_context(add, all_years)
print(context_got)

['', '']


In [54]:
def find_year_of_manifacture(contexts):
    import re
    for text in contexts:
        test = [m.start() for m in re.finditer('egist', text)]
        if len(test) > 0:
            year = 
    return km_number    
find_year_of_manifacture(context_got)

['76699', '2', '2018', '107604']

In [51]:
def find_km(contexts):
    import re
    for text in contexts:
        test = [m.start() for m in re.finditer('Km', text)]
        if len(test) > 0:
            km_number = get_numbers_from_text(text)
    return km_number    
find_km(context_got)

['202160']

In [43]:
def get_context(text, list_of_tokens, context_span=20):
    import re
    context = []
    all_indexes = []
    for token in list_of_tokens:
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        all_occurences_indices_ends = [m.end() for m in re.finditer(token, text)]
        
        if len(all_occurences_indices) > 1:
            all_occurences_indices = [max(all_occurences_indices)] #have to solve more sophistically
        if len(all_occurences_indices_ends) > 1:
            all_occurences_indices_ends = [max(all_occurences_indices_ends)]
        all_indexes.append(all_occurences_indices_ends)
        print(max(all_indexes))
        print(all_indexes[-1])
        
        for index in all_occurences_indices:
            print(index)
            left_index = max(min(index - context_span, index - all_indexes[-1]), 0)
            #left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            
            if left_index == 0:
                substring = ' '.join(substring.split(' ')[:-1])
            elif right_index == len(text):
                substring = ' '.join(substring.split(' ')[1:])
            else:
                substring = ' '.join(substring.split(' ')[1:-1])
            context.append(substring)
    return context, all_indexes
get_context(add, all_numbers)

[24]
[24]
21


TypeError: unsupported operand type(s) for -: 'int' and 'list'